# Exercise 5 - Actor Handles

**GOAL:** The goal of this exercise is to show how to pass around actor handles.

Suppose we wish to have multiple tasks invoke methods on the same actor. For example, we may have a single actor that records logging information from a number of tasks. We can achieve this by passing a handle to the actor as an argument into the relevant tasks.

### Concepts for this Exercise - Actor  Handles

First of all, suppose we've created an actor as follows.

```python
@ray.remote
class Actor(object):
    def method(self):
        pass

# Create the actor
actor = Actor.remote()
```

Then we can define a remote function (or another actor) that takes an actor handle as an argument.

```python
@ray.remote
def f(actor):
    # We can invoke methods on the actor.
    x_id = actor.method.remote()
    # We can block and get the results.
    return ray.get(x_id)
```

Then we can invoke the remote function a few times and pass in the actor handle.

```python
# Each of the three tasks created below will invoke methods on the same actor.
f.remote(actor)
f.remote(actor)
f.remote(actor)
```

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import ray
import time

ray.init(num_cpus=4, ignore_reinit_error=True)  # include_webui=False, 

2021-02-02 16:33:10,945	INFO services.py:1173 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '10.0.0.58',
 'raylet_ip_address': '10.0.0.58',
 'redis_address': '10.0.0.58:6379',
 'object_store_address': 'tcp://127.0.0.1:61803',
 'raylet_socket_name': 'tcp://127.0.0.1:62409',
 'webui_url': '127.0.0.1:8265',
 'session_dir': 'C:\\Users\\Eugene\\AppData\\Local\\Temp\\ray\\session_2021-02-02_16-33-09_420460_23404',
 'metrics_export_port': 62178,
 'node_id': '6193e533cc9e1ca76a410b431f153e3057813bbc'}

In this exercise, we're going to write some code that runs several "experiments" in parallel and has each experiment log its results to an actor. The driver script can then periodically pull the results from the logging actor.

**EXERCISE:** Turn this `LoggingActor` class into an actor class.

In [2]:
from collections import defaultdict

@ray.remote
class LoggingActor(object):
    def __init__(self):
        self.logs = defaultdict(lambda: [])
    
    def log(self, index, message):
        self.logs[index].append(message)
    
    def get_logs(self):
        return dict(self.logs)


assert hasattr(LoggingActor, 'remote'), ('You need to turn LoggingActor into an '
                                         'actor (by using the ray.remote keyword).')

**EXERCISE:** Instantiate the actor.

In [3]:
logging_actor = LoggingActor.remote()

# Some checks to make sure this was done correctly.
assert hasattr(logging_actor, 'get_logs')

Now we define a remote function that runs and pushes its logs to the `LoggingActor`.

**EXERCISE:** Modify this function so that it invokes methods correctly on `logging_actor` (you need to change the way you call the `log` method).

In [4]:
@ray.remote
def run_experiment(experiment_index, logging_actor):
    for i in range(60):
        time.sleep(1)
        # Push a logging message to the actor.
        logging_actor.log.remote(experiment_index, 'On iteration {}'.format(i))

Now we create several tasks that use the logging actor.

In [5]:
experiment_ids = [run_experiment.remote(i, logging_actor) for i in range(3)]

While the experiments are running in the background, the driver process (that is, this Jupyter notebook) can query the actor to read the logs.

**EXERCISE:** Modify the code below to dispatch methods to the `LoggingActor`.

In [6]:
logs = ray.get(logging_actor.get_logs.remote())

assert isinstance(logs, dict), ("Make sure that you dispatch tasks to the "
                                "actor using the .remote keyword and get the results using ray.get.")
logs  # constantly updating

{1: ['On iteration 0',
  'On iteration 1',
  'On iteration 2',
  'On iteration 3',
  'On iteration 4'],
 2: ['On iteration 0',
  'On iteration 1',
  'On iteration 2',
  'On iteration 3',
  'On iteration 4'],
 0: ['On iteration 0',
  'On iteration 1',
  'On iteration 2',
  'On iteration 3',
  'On iteration 4']}

In [7]:
%%time
# trying to extract all the values before gathering
from functools import reduce

ray.get(experiment_ids)
reduce(lambda x, y: x + y, ray.get(logging_actor.get_logs.remote()).values())
# approximately 180 seconds since everything is serial but not really

Wall time: 53.5 s


['On iteration 0',
 'On iteration 1',
 'On iteration 2',
 'On iteration 3',
 'On iteration 4',
 'On iteration 5',
 'On iteration 6',
 'On iteration 7',
 'On iteration 8',
 'On iteration 9',
 'On iteration 10',
 'On iteration 11',
 'On iteration 12',
 'On iteration 13',
 'On iteration 14',
 'On iteration 15',
 'On iteration 16',
 'On iteration 17',
 'On iteration 18',
 'On iteration 19',
 'On iteration 20',
 'On iteration 21',
 'On iteration 22',
 'On iteration 23',
 'On iteration 24',
 'On iteration 25',
 'On iteration 26',
 'On iteration 27',
 'On iteration 28',
 'On iteration 29',
 'On iteration 30',
 'On iteration 31',
 'On iteration 32',
 'On iteration 33',
 'On iteration 34',
 'On iteration 35',
 'On iteration 36',
 'On iteration 37',
 'On iteration 38',
 'On iteration 39',
 'On iteration 40',
 'On iteration 41',
 'On iteration 42',
 'On iteration 43',
 'On iteration 44',
 'On iteration 45',
 'On iteration 46',
 'On iteration 47',
 'On iteration 48',
 'On iteration 49',
 'On itera

2021-02-02 16:35:46,649	WARNING worker.py:1034 -- The dashboard on node DESKTOP-6EJMNGA failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Eugene\anaconda3\envs\CS6475\lib\site-packages\ray\new_dashboard\dashboard.py", line 192, in <module>
    log_dir=args.log_dir)
  File "C:\Users\Eugene\anaconda3\envs\CS6475\lib\site-packages\ray\new_dashboard\dashboard.py", line 81, in __init__
    build_dir = setup_static_dir()
  File "C:\Users\Eugene\anaconda3\envs\CS6475\lib\site-packages\ray\new_dashboard\dashboard.py", line 45, in setup_static_dir
    "&& npm run build)", build_dir)
FileNotFoundError: [Errno 2] Dashboard build directory not found. If installing from source, please follow the additional steps required to build the dashboard(cd python/ray/new_dashboard/client && npm install && npm ci && npm run build): 'C:\\Users\\Eugene\\anaconda3\\envs\\CS6475\\lib\\site-packages\\ray\\new_dashboard\\client\\build'

(pid=None) [2021-02-02 16:35:49,640 C 16908 

(pid=None) [2021-02-02 16:36:33,469 C 19880 13328] redis_client.cc:74:  Check failed: num_attempts < RayConfig::instance().redis_db_connect_retries() Expected 1 Redis shard addresses, found 3
(pid=None) [2021-02-02 16:36:33,472 E 19880 13328] logging.cc:414: *** Aborted at 1612301793 (unix time) try "date -d @1612301793" if you are using GNU date ***
(pid=None) [2021-02-02 16:36:33,480 E 19880 13328] logging.cc:414:     @     0x7ff8f556caad raise
(pid=None) [2021-02-02 16:36:33,480 E 19880 13328] logging.cc:414:     @     0x7ff8f556dab1 abort
(pid=None) [2021-02-02 16:36:33,483 E 19880 13328] logging.cc:414:     @     0x7ff628a3941b public: void __cdecl google::NullStreamFatal::`vbase destructor'(void) __ptr64
(pid=None) [2021-02-02 16:36:33,483 E 19880 13328] logging.cc:414:     @     0x7ff628a37dd1 public: virtual __cdecl google::NullStreamFatal::~NullStreamFatal(void) __ptr64
(pid=None) [2021-02-02 16:36:33,483 E 19880 13328] logging.cc:414:     @     0x7ff628902ef4 public: class go

**EXERCISE:** Try running the above box multiple times and see how the results change (while the experiments are still running in the background). You can also try running more of the experiment tasks and see what happens.